In [11]:
import json
import os
from tqdm import tqdm
from PIL import Image
from PIL import ImageDraw
import json
from multiprocessing import Pool

In [12]:
data_split = ['train','val']

path = '/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation/'

In [13]:
fl = open('label2id.json')
label2id = json.load(fl)

fl.close()

In [14]:
fl = open('id2label.json')
id2label = json.load(fl)
fl.close()

In [15]:
def process_single_file(args):
    
    each_file,each_folder,combined_folder_path,new_path = args
    
    fl = open(combined_folder_path+'/'+each_file)
    json_obj = json.load(fl)
    fl.close()

    json_data = json_obj
    img_height = json_data["imgHeight"]
    img_width = json_data["imgWidth"]

    # Create a blank image
    image = Image.new(mode="L",size= (img_width, img_height))
    draw = ImageDraw.Draw(image)

    # Draw polygons on the image
    for obj in json_data["objects"]:
        if obj["draw"]:
            label = obj["label"]
            correct_id = label2id[label]
            polygon = [(point[0], point[1]) for point in obj["polygon"]]
            draw.polygon(polygon,fill=correct_id)

    image.save(new_path+'/'+each_folder+'/'+each_file.replace("json","png"))
    
    print("\nprocessed image ",new_path+'/'+each_folder+'/'+each_file.replace("json","png"))

In [16]:


def preprocess_annotations(split):
    
    combined_path = path+'gtFine/'+split+'/'
    
    new_path = path+'/label_processed'+'/'+split
    
    folders = os.listdir(combined_path)
    
    try:
        os.mkdir(new_path)
    except Exception as e:
        print(e)
        print(new_path, " already exist")

    for each_folder in folders:
        
        combined_folder_path = combined_path+'/'+each_folder

        files = os.listdir(combined_folder_path)
        
        try:
            os.mkdir(new_path+'/'+each_folder)
        except:
            print(new_path+'/'+each_folder," already exist")

        
        pool_args = []
        
        for each_file in files:
            
            pool_args.append([each_file,each_folder,combined_folder_path,new_path])
            
        pool = Pool(processes=20)
        pool.map(process_single_file,pool_args)
        
        
            
            

In [17]:
for each in data_split:
    preprocess_annotations(each)

[Errno 17] File exists: '/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train'
/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train  already exist
/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/69  already exist

processed image 
processed image   /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/69/000733_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/69/023418_gtFine_polygons.png


processed image 
processed image 
processed image 
processed image 
processed image  
processed image 
processed image  
processed image  
processed image  
processed image  /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/69/020416_gtFine_polygons.png /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/69/016309_gtFine_polygons.png 
processed image /scratch/j/jcaunedo/umar1/segmentation/I

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171  already exist

processed image 
processed image 
processed image 
processed image 
processed image 
processed image 
processed image  
processed image  
processed image 
processed image 
processed image      
processed image /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171/941223_gtFine_polygons.png
processed image  /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171/242459_gtFine_polygons.png 
processed image  /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171/003222_gtFine_polygons.png /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171/238765_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171/245059_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/171/243731_gtFine_polygons.pn

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)





processed image  /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/716423_gtFine_polygons.png

processed image 
processed image 
processed image   /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/033872_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/499306_gtFine_polygons.png
 

processed image /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/494821_gtFine_polygons.png
processed image   /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/497236_gtFine_polygons.png
/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/500093_gtFine_polygons.png

processed image 

processed image   
processed image 
processed image /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/11/222220_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//la

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 
processed image 

/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/139/002251_gtFine_polygons.png
 /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/139/002634_gtFine_polygons.png
processed image 
 
processed image /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/139/022963_gtFine_polygons.png 
/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/139/020074_gtFine_polygons.png
/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/135  already exist

processed image 
processed image 
processed image 
processed image     
processed image 
processed image 
processed image /scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/135/690395_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_processed/train/135/066356_gtFine_polygons.png/scratch/j/jcaunedo/umar1/segmentation/IDD_Segmentation//label_p